# Data Analysis

In [622]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import math as math
import numpy as np
import re

## 1. Clean and Prepare Data

### 1.1. Data Overview

In [623]:
df = pd.read_csv('../data/albert_heijn/ah_data.csv')
df_original = df.copy() # keep copy of original dataframe
print('Number of rows before data cleaning: ' + str(len(df)))

Number of rows before data cleaning: 17072


In [624]:
df.head()

,product,url,category,price,amount,content,unit,vegan,vegetarian,nutriscore,carbs_100g,protein_100g,fat_100g,alcohol_100g
0,De Zaanse Hoeve Roomboter ongezouten,https://www.ah.nl/producten/product/wi429615/d...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,2.49,250.0,Inhoud en gewicht250 GramPortiegrootte: 5 gram...,g,False,True,E,1.0,0.6,82.0,NaN
1,AH Halvarine lekker op brood,https://www.ah.nl/producten/product/wi127489/a...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,1.09,500.0,Inhoud en gewicht500 GramPortiegrootte: 5 gram...,g,True,True,C,0.6,0.0,39.0,NaN
2,AH Zuivelspread naturel,https://www.ah.nl/producten/product/wi506228/a...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,0.79,200.0,Inhoud en gewicht200 GramPortiegrootte: 15 gra...,g,False,True,D,3.3,7.8,16.0,NaN
3,AH Luchtige cracker volkoren met rijstbloem,https://www.ah.nl/producten/product/wi196377/a...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,0.99,250.0,"Inhoud en gewicht250 GramPortiegrootte: 7,5 gr...",g,True,True,A,63.0,10.0,3.6,NaN
4,AH Smeerkaas naturel,https://www.ah.nl/producten/product/wi249/ah-s...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,0.79,100.0,Inhoud en gewicht100 GramPortiegrootte: 15 gra...,g,False,False,D,2.4,14.0,21.0,NaN


In [625]:
# Number of NAs per field:
df.isna().sum()

product             0
url                 0
category            0
price               7
amount           1489
content           441
unit             1489
vegan               0
vegetarian          0
nutriscore       9364
carbs_100g        932
protein_100g      934
fat_100g          953
alcohol_100g    16948
dtype: int64

In [626]:
# Number of Products per Category
df.groupby(["category"])["product"].count()

category
aardappel-groente-fruit?minPrice=0&maxPrice=2.99             874
aardappel-groente-fruit?minPrice=3&maxPrice=99               316
bakkerij-en-banket                                           981
diepvries                                                    861
kaas-vleeswaren-tapas?minPrice=0&maxPrice=2.99               849
kaas-vleeswaren-tapas?minPrice=3&maxPrice=99                 627
ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60              886
ontbijtgranen-en-beleg?minPrice=2.61&maxPrice=99             966
pasta-rijst-en-wereldkeuken?minPrice=0&maxPrice=1.99         796
pasta-rijst-en-wereldkeuken?minPrice=2&maxPrice=3.99         993
pasta-rijst-en-wereldkeuken?minPrice=4&maxPrice=99           233
salades-pizza-maaltijden                                     655
snoep-koek-chips-en-chocolade?minPrice=0&maxPrice=1.89       926
snoep-koek-chips-en-chocolade?minPrice=1.90&maxPrice=2.59    940
snoep-koek-chips-en-chocolade?minPrice=2.60&maxPrice=99      876
soepen-sauzen-kr

### 1.2 Remove Duplicate Products

In [627]:
print('Number of rows before deleting duplicates: ' + str(len(df)))
df = df.drop_duplicates('url', keep='last')
print('Number of rows after deleting duplicates: ' + str(len(df)))

Number of rows before deleting duplicates: 17072
Number of rows after deleting duplicates: 11852


### 1.3. Estimate amount for missing or inexact amount values based on 'content' label

In [628]:
def get_content_amount(content):
    if str(content) == "nan":
        amount = np.NaN
        content_type = "1: Content = NaN"
    else:
        content = content.lower()
        if re.search('gewicht[\d]* gram',content) and content.lower().find('liter')==-1:
            # get total amount label ('gewicht'). Does not work for liquids (ml or liters)
            amount = re.search('gewicht[\d]* gram',content)[0]
            amount = amount.rstrip(' gram').strip('gewicht').replace(',','.')
            amount = float(amount)
            content_type = "2: Total weight in g"
        elif re.search('portiegrootte: [\d\,]* gram',content) and re.search('antal porties: [\d\,]*',content):
            # get portion size and nr of portions
            portion_size = re.search('portiegrootte: [\d\,]* gram',content)[0]
            portion_size = portion_size.rstrip(' gram').strip('portiegrootte: ').replace(',','.')
            portion_size = float(portion_size)
            number_of_portions = re.search('antal porties: [\d\,]*',content)[0]
            number_of_portions = number_of_portions.strip('antal porties: ')
            number_of_portions = float(number_of_portions)
            amount = portion_size * number_of_portions
            content_type = "3: portion_size * number_of_portions"
        else:
            amount = np.NaN
            content_type = "4: NaN (Unknown content format)"
    return amount, content_type

In [629]:
def amount_not_exact(amount):
    if str(amount) == 'nan':
        not_exact = True
    elif amount > 6.5 and amount - math.floor(amount)>0:
        # if number has decimals then amount value was not accurate enough
        # add 6.5 limit to exclude small products like spices
        not_exact = True
    else:
        not_exact = False
    return not_exact

In [630]:
# Estimate amounts based on label
df['amount_old'] = df['amount'] # create copy of amount column
df['amount_content'] = df['content'].apply(lambda x: get_content_amount(x)[0])
df['content_type'] = df['content'].apply(lambda x: get_content_amount(x)[1])
print("Types of 'Content' for Products with no Amount:")
df[df['unit'].isna()].groupby(["content_type"]).count()["product"]
# We are able to estimate more than 50% of missing amounts using the content label (types 2 and 3)

Types of 'Content' for Products with no Amount:


content_type
1: Content = NaN                        276
2: Total weight in g                    370
3: portion_size * number_of_portions    142
4: NaN (Unknown content format)         253
Name: product, dtype: int64

In [631]:
print('Number of products with inexact amounts (either missing or poorly estimated):')
print(sum([amount_not_exact(x) for x in df['amount']]))

Number of products with inexact amounts (either missing or poorly estimated):
1365


In [632]:
# Change amounts that are not exact AND have a valid estimate from the 'content' label
amounts_to_change = [amount_not_exact(x) for x in df['amount']] & df['amount_content'].notna()
print("Amounts before being changed:")
df[amounts_to_change]['amount']

Amounts before being changed:


21       132.352941
427             NaN
918      121.111111
924      121.428571
943             NaN
            ...    
17067           NaN
17068           NaN
17069           NaN
17070           NaN
17071           NaN
Name: amount, Length: 679, dtype: float64

In [633]:
df.loc[amounts_to_change, 'amount'] = df[amounts_to_change]['amount_content'] # Update amounts
df.loc[amounts_to_change, 'unit'] = 'g' # Update unit

In [634]:
print("Amounts after being changed:")
df[amounts_to_change]['amount']
# amount now have the exact values

Amounts after being changed:


21        130.0
427       200.0
918       120.0
924       120.0
943       100.0
          ...  
17067     146.0
17068     202.0
17069     182.0
17070     212.0
17071    1030.0
Name: amount, Length: 679, dtype: float64

In [635]:
print("Number of products which still have missing amounts (these will have to be deleted):")
print(sum(df['unit'].isna()))

Number of products which still have missing amounts (these will have to be deleted):
529


### 1.4 Remove/Update NAs

In [636]:
# Drop records where amount could not be estimated
df = df.dropna(subset=['unit'])
print('Number of rows before data cleaning: ' + str(len(df)))

Number of rows before data cleaning: 11323


In [637]:
print('Number of products with inexact amounts:')
print(sum([amount_not_exact(x) for x in df['amount']]))

Number of products with inexact amounts:
158


In [638]:
# Round remaining inexact amounts. In most cases this results in the correct amount:
df.loc[[amount_not_exact(x) for x in df['amount']], 'amount'] = [round(x) for x in df[[amount_not_exact(x) for x in df['amount']]]['amount']]

In [639]:
print('Number of products with inexact amounts:')
print(sum([amount_not_exact(x) for x in df['amount']]))

Number of products with inexact amounts:
0


In [640]:
# Fill NAs of numeric columns with zero
df['carbs_100g'] = df['carbs_100g'].fillna(0)
df['protein_100g'] = df['protein_100g'].fillna(0)
df['fat_100g'] = df['fat_100g'].fillna(0)
df['alcohol_100g'] = df['alcohol_100g'].fillna(0)

### 1.5 Add New Fields

In [641]:
# Calculate calories: 4 kcal per g of protein/carbs, 7 kcal per g of alcohol, 9 kcal per g of fat
df['calories_100g'] = df.apply(lambda x: x['carbs_100g'] * 4 + x['protein_100g'] * 4 + x['fat_100g'] * 9 + x['alcohol_100g'] * 7, axis = 1)

In [642]:
# Add total amount of macros
df['calories'] = df['calories_100g']*df['amount']/100
df['protein'] = df['protein_100g']*df['amount']/100
df['carbs'] = df['carbs_100g']*df['amount']/100
df['fat'] = df['fat_100g']*df['amount']/100
df['alcohol'] = df['alcohol_100g']*df['amount']/100

In [643]:
df

,product,url,category,price,amount,content,unit,vegan,vegetarian,nutriscore,...,alcohol_100g,amount_old,amount_content,content_type,calories_100g,calories,protein,carbs,fat,alcohol
5,AH Volkoren havermout,https://www.ah.nl/producten/product/wi48405/ah...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,0.69,500.0,Inhoud en gewicht500 GramPortiegrootte: 40 gra...,g,False,False,NaN,...,0.0,500.000000,500.0,2: Total weight in g,355.0,1775.000,70.000,295.00,35.00,0.0
6,AH Beschuit volkoren,https://www.ah.nl/producten/product/wi1625/ah-...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,0.49,140.0,Inhoud en gewicht140 GramPortiegrootte: 11 gra...,g,False,True,A,...,0.0,140.000000,140.0,2: Total weight in g,380.6,532.840,21.000,95.20,7.56,0.0
21,AH Beschuit naturel,https://www.ah.nl/producten/product/wi1624/ah-...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,0.45,130.0,Inhoud en gewicht130 GramPortiegrootte: 10 gra...,g,False,True,A,...,0.0,132.352941,130.0,2: Total weight in g,410.8,534.040,19.500,98.80,6.76,0.0
24,AH Honing,https://www.ah.nl/producten/product/wi479788/a...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,1.99,350.0,Inhoud en gewicht350 GramPortiegrootte: 15 gra...,g,False,True,D,...,0.0,350.000000,350.0,2: Total weight in g,325.6,1139.600,1.400,283.50,0.00,0.0
34,Brinta Volkoren graanontbijt,https://www.ah.nl/producten/product/wi166916/b...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,2.09,500.0,Inhoud en gewicht500 GramAantal porties: 14,g,False,False,NaN,...,0.0,500.000000,500.0,2: Total weight in g,329.1,1645.500,70.000,320.00,9.50,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17067,SUNT Donut lemon poppyseed,https://www.ah.nl/producten/product/wi537024/s...,bakkerij-en-banket,2.99,146.0,Inhoud en gewicht146 GramPortiegrootte: portie,g,False,False,NaN,...,0.0,NaN,146.0,2: Total weight in g,332.0,484.720,5.110,70.08,20.44,0.0
17068,AH Karamel zeezout cheesecake,https://www.ah.nl/producten/product/wi537027/a...,bakkerij-en-banket,4.00,202.0,Inhoud en gewicht2 StuksPortiegrootte: 101 gra...,g,False,True,E,...,0.0,NaN,202.0,3: portion_size * number_of_portions,337.6,681.952,9.898,78.78,36.36,0.0
17069,AH Cookie & cream slofjes,https://www.ah.nl/producten/product/wi537030/a...,bakkerij-en-banket,3.50,182.0,Inhoud en gewicht2 StuksPortiegrootte: 91 gram...,g,False,True,E,...,0.0,NaN,182.0,3: portion_size * number_of_portions,447.6,814.632,7.098,81.90,50.96,0.0
17070,AH Salted caramel slofjes,https://www.ah.nl/producten/product/wi537031/a...,bakkerij-en-banket,3.50,212.0,Inhoud en gewicht212 GramPortiegrootte: 106 gr...,g,False,False,E,...,0.0,NaN,212.0,2: Total weight in g,354.2,750.904,6.996,80.56,44.52,0.0


### Delete Outliers

#### These 

## 2. Analyze Data

In [647]:
# Price per g of each macro
df['price_g_calories'] = df['price']/df['calories']
df['price_g_protein'] = df['price']/df['protein']
df['price_g_carbs'] = df['price']/df['carbs']
df['price_g_fat'] = df['price']/df['fat']

In [649]:
# Determine percentage of macro relative to total amount of calories:
df['perc_calories_protein'] = df['protein']*4/df['calories']
# df['perc_calories_carbs'] = df['price']/df['calories']
# df['perc_calories_fat'] = df['price']/df['calories']
# df['perc_calories_alcohol'] = df['price']/df['calories']

In [651]:
df.sort_values(by=['price_g_protein']).iloc[1:40]

,product,url,category,price,amount,content,unit,vegan,vegetarian,nutriscore,...,calories,protein,carbs,fat,alcohol,price_g_calories,price_g_protein,price_g_carbs,price_g_fat,perc_calories_protein
16219,AH Zelfrijzend bakmeel,https://www.ah.nl/producten/product/wi63490/ah...,bakkerij-en-banket,0.99,1000.0,Inhoud en gewicht1 KilogramPortiegrootte: 100 ...,g,True,True,C,...,3335.000,120.00,680.00,15.000,0.0,0.000297,0.008250,0.001456,0.066000,0.143928
16924,AH Patentbloem,https://www.ah.nl/producten/product/wi951/ah-p...,bakkerij-en-banket,0.85,1000.0,Inhoud en gewicht1 KilogramPortiegrootte: 100 ...,g,True,True,A,...,3370.000,100.00,720.00,10.000,0.0,0.000252,0.008500,0.001181,0.085000,0.118694
6725,AH Paneermeel naturel broodkruim,https://www.ah.nl/producten/product/wi196776/a...,pasta-rijst-en-wereldkeuken?minPrice=0&maxPric...,0.49,400.0,Inhoud en gewicht400 GramPortiegrootte: 15 gra...,g,True,True,A,...,1385.600,52.00,280.00,6.400,0.0,0.000354,0.009423,0.001750,0.076562,0.150115
16344,AH Volkoren tarwemeel,https://www.ah.nl/producten/product/wi47109/ah...,bakkerij-en-banket,1.15,1000.0,Inhoud en gewicht1 KilogramPortiegrootte: 100 ...,g,True,True,A,...,3202.000,120.00,640.00,18.000,0.0,0.000359,0.009583,0.001797,0.063889,0.149906
16188,AH Patent tarwebloem,https://www.ah.nl/producten/product/wi952/ah-p...,bakkerij-en-banket,0.49,500.0,Inhoud en gewicht500 GramPortiegrootte: 100 gr...,g,True,True,A,...,1685.000,50.00,360.00,5.000,0.0,0.000291,0.009800,0.001361,0.098000,0.118694
16378,AH Zelfrijzend bakmeel,https://www.ah.nl/producten/product/wi953/ah-z...,bakkerij-en-banket,0.59,500.0,Inhoud en gewicht500 GramPortiegrootte: 100 gr...,g,True,True,C,...,1663.000,60.00,340.00,7.000,0.0,0.000355,0.009833,0.001735,0.084286,0.144317
284,AH Volkoren havervlokken,https://www.ah.nl/producten/product/wi509276/a...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,0.69,500.0,Inhoud en gewicht500 GramPortiegrootte: 40 gra...,g,True,True,A,...,1775.000,70.00,295.00,35.000,0.0,0.000389,0.009857,0.002339,0.019714,0.157746
5,AH Volkoren havermout,https://www.ah.nl/producten/product/wi48405/ah...,ontbijtgranen-en-beleg?minPrice=0&maxPrice=2.60,0.69,500.0,Inhoud en gewicht500 GramPortiegrootte: 40 gra...,g,False,False,NaN,...,1775.000,70.00,295.00,35.000,0.0,0.000389,0.009857,0.002339,0.019714,0.157746
10481,Valle del sole Groene spliterwten,https://www.ah.nl/producten/product/wi198415/v...,soepen-sauzen-kruiden-olie?minPrice=1.80&maxPr...,1.99,900.0,Inhoud en gewicht900 Gram,g,False,False,NaN,...,3150.000,198.00,549.00,18.000,0.0,0.000632,0.010051,0.003625,0.110556,0.251429
11184,Valle del sole Gele spliterwten,https://www.ah.nl/producten/product/wi198409/v...,soepen-sauzen-kruiden-olie?minPrice=1.80&maxPr...,1.99,900.0,Inhoud en gewicht900 Gram,g,False,False,NaN,...,2898.000,198.00,486.00,18.000,0.0,0.000687,0.010051,0.004095,0.110556,0.273292
